In [1]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from PIL import Image

import torch
import torchvision
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torchvision import transforms, utils
import random
from torchvision.transforms import Resize, ToTensor, Normalize
from torch.utils.data import Dataset, DataLoader, random_split, SubsetRandomSampler, WeightedRandomSampler

In [2]:
def seed_everything(seed):
    """
    동일한 조건으로 학습을 할 때, 동일한 결과를 얻기 위해 seed를 고정시킵니다.
    
    Args:
        seed: seed 정수값
    """
    torch.manual_seed(seed) # pytorch의 random seed 고정
    torch.cuda.manual_seed(seed) # GPU 에서 사용하는 난수 생성 시드 고정
    # torch.cuda.manual_seed_all(seed)  # if use multi-GPU
    torch.backends.cudnn.deterministic = True # CuDNN 부분고정
    torch.backends.cudnn.benchmark = False # CuDNN 부분고정
    np.random.seed(seed) # Numpy 부분
    random.seed(seed) # transforms에서 random 라이브러리를 사용하기 때문에 random 라이브러리를 불러서 고정

seed_everything(42)

In [3]:
# from zipfile import ZipFile

# file_name = "zipfiles/scenery.zip"

# with ZipFile(file_name, 'r') as zip:
#     zip.extractall("scenery") 
#     print('File is unzipped in vector folder')

In [4]:
def search(dirname, result):  # 하위목록의 모든 파일을 찾는 함수
    try:
        filenames = os.listdir(dirname)
        for filename in filenames:
            full_filename = os.path.join(dirname, filename)
            if os.path.isdir(full_filename):
                search(full_filename, result)
            else:
                ext = os.path.splitext(full_filename)[-1]  # 확장자 체크
                if ext:
                    result.append(full_filename)
    except PermissionError:
        pass

scenery_all_path = []
search("scenery", scenery_all_path)
vector_all_path = []
search("vector", vector_all_path)

In [5]:
len(scenery_all_path), len(vector_all_path)

(22125, 14876)

In [6]:
vector_all_path[:5]

['vector/moped-153275__340.png',
 'vector/cake-305217__340.png',
 'vector/bird-1296169__480.png',
 'vector/lady-2860835__340.png',
 'vector/bush-575517__340.png']

In [7]:
def labeling(dirname, result, prefix):  # 라벨링하는 함수
    try:
        filenames = os.listdir(dirname)
        for filename in filenames:
            keyword = prefix + filename.split(".")[0].split("__")[0].split("-")[0]
            result.append(keyword)
    except PermissionError:
        pass

In [8]:
dirname, vector_label = "vector", []
labeling(dirname, vector_label, "a vector image of ")
dirname, scenery_label = "scenery", []
labeling(dirname, scenery_label, "a scenery of ")

In [9]:
len(vector_label), len(scenery_label)

(14876, 22125)

In [10]:
scenery_df = pd.DataFrame(scenery_all_path, columns = ['path'])

scenery_df['label'] = scenery_label
scenery_df

,path,label
0,scenery/tea-garden-2823303__340.png,a scenery of tea
1,scenery/grain-field-6571349__340.png,a scenery of grain
2,scenery/santa-barbara-1584103__340.png,a scenery of santa
3,scenery/field-6772703__340.png,a scenery of field
4,scenery/cow-4960242__340.png,a scenery of cow
...,...,...
22120,scenery/tajikistan-4582152__340.png,a scenery of tajikistan
22121,scenery/river-6748978__340.png,a scenery of river
22122,scenery/mountains-5729187__340.png,a scenery of mountains
22123,scenery/spoon-bills-5791480__340.png,a scenery of spoon


In [11]:
vector_df = pd.DataFrame(vector_all_path, columns = ['path'])

vector_df['label'] = vector_label
vector_df

,path,label
0,vector/moped-153275__340.png,a vector image of moped
1,vector/cake-305217__340.png,a vector image of cake
2,vector/bird-1296169__480.png,a vector image of bird
3,vector/lady-2860835__340.png,a vector image of lady
4,vector/bush-575517__340.png,a vector image of bush
...,...,...
14871,vector/sos-2753818__340.png,a vector image of sos
14872,vector/silhouette-1229849__340.png,a vector image of silhouette
14873,vector/dove-7100089__340.png,a vector image of dove
14874,vector/couple-2029712__340.png,a vector image of couple


In [12]:
df = pd.concat([vector_df, scenery_df], ignore_index=True)
df

,path,label
0,vector/moped-153275__340.png,a vector image of moped
1,vector/cake-305217__340.png,a vector image of cake
2,vector/bird-1296169__480.png,a vector image of bird
3,vector/lady-2860835__340.png,a vector image of lady
4,vector/bush-575517__340.png,a vector image of bush
...,...,...
36996,scenery/tajikistan-4582152__340.png,a scenery of tajikistan
36997,scenery/river-6748978__340.png,a scenery of river
36998,scenery/mountains-5729187__340.png,a scenery of mountains
36999,scenery/spoon-bills-5791480__340.png,a scenery of spoon


In [13]:
normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                 std=[0.229, 0.224, 0.225])

data_transforms = {
    'train':
    transforms.Compose([
        transforms.Resize((224,224)),
        transforms.ToTensor(),
        normalize
    ]),
    'val':
    transforms.Compose([
        transforms.Resize((224,224)),
        transforms.ToTensor(),
        normalize
    ]),
}

In [14]:
class CustomDataset(Dataset):
    def __init__(self, img_paths_label, transform):
        self.X = img_paths_label['path']
        self.y = img_paths_label['label']
        self.transform = transform
    
    def __getitem__(self, index):
        # image = Image.open(self.X.iloc[index])
        image = Image.open(self.X.iloc[index]).convert('RGB')
        label = self.y.iloc[index]
        
        if self.transform:
            image = self.transform(image)
        # return image, torch.tensor(label)
        return image, label
    
    def __len__(self):
        return len(self.X)

In [15]:
from sklearn.model_selection import train_test_split

train, valid = train_test_split(df, test_size=0.2,
                               shuffle=True,
                               random_state=42)

In [16]:
BATCH_SIZE = 32

train_dataset = CustomDataset(train, data_transforms['train'])
train_dataloader = DataLoader(train_dataset,
                             batch_size = BATCH_SIZE,
                             shuffle=True)

valid_dataset = CustomDataset(valid, data_transforms['val'])
valid_dataloader = DataLoader(valid_dataset,
                             batch_size=BATCH_SIZE,
                             shuffle=True)

In [21]:
next(iter(train_dataloader))[0]

tensor([[[[-2.1179, -2.1179, -2.1179,  ..., -2.1179, -2.1179, -2.1179],
          [-2.1179, -2.1179, -2.1179,  ..., -2.1179, -2.1179, -2.1179],
          [-2.1179, -2.1179, -2.1179,  ..., -2.1179, -2.1179, -2.1179],
          ...,
          [-2.1179, -2.1179, -2.1179,  ..., -2.1179, -2.1179, -2.1179],
          [-2.1179, -2.1179, -2.1179,  ..., -2.1179, -2.1179, -2.1179],
          [-2.1179, -2.1179, -2.1179,  ..., -2.1179, -2.1179, -2.1179]],

         [[-2.0357, -2.0357, -2.0357,  ..., -2.0357, -2.0357, -2.0357],
          [-2.0357, -2.0357, -2.0357,  ..., -2.0357, -2.0357, -2.0357],
          [-2.0357, -2.0357, -2.0357,  ..., -2.0357, -2.0357, -2.0357],
          ...,
          [-2.0357, -2.0357, -2.0357,  ..., -2.0357, -2.0357, -2.0357],
          [-2.0357, -2.0357, -2.0357,  ..., -2.0357, -2.0357, -2.0357],
          [-2.0357, -2.0357, -2.0357,  ..., -2.0357, -2.0357, -2.0357]],

         [[-1.8044, -1.8044, -1.8044,  ..., -1.8044, -1.8044, -1.8044],
          [-1.8044, -1.8044, -

In [22]:
next(iter(train_dataloader))[0].shape

torch.Size([32, 3, 224, 224])

* 간혹 `unidentifiedimageerror: cannot identify image file` 에러 발생